## Low-Rank Autoregressive Tensor Completion (LATC)

This notebook shows how to implement a LATC (with truncated nuclear norm) imputer on some real-world traffic data sets. To overcome the problem of missing values within multivariate time series data, this method takes into account both low-rank structure and time series regression. For an in-depth discussion of LATC, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Mengying Lei, Nicolas Saunier, Lijun Sun (2021). <b>Low-Rank Autorgressive Tensor Completion for Spatiotemporal Traffic Data Imputation</b>. arXiv:2104.14936. <a href="https://arxiv.org/abs/2104.14936" title="PDF"><b>[PDF]</b></a> 
</font>
</div>


### Define LATC-imputer kernel

We start by introducing some necessary functions that relies on `Numpy`.

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>ten2mat</code>:</b> <font color="black">Unfold tensor as matrix by specifying mode.</font></li>
<li><b><code>mat2ten</code>:</b> <font color="black">Fold matrix as tensor by specifying dimension (i.e, tensor size) and mode.</font></li>
<li><b><code>svt_tnn</code>:</b> <font color="black">Implement the process of Singular Value Thresholding (SVT).</font></li>
</ul>
</div>

In [1]:
import numpy as np

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [2]:
def svt_tnn(mat, tau, theta):
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = 0)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[: theta] = 1
        mid[theta : idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:, : idx] @ np.diag(mid)) @ (u[:, : idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, tau, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[: idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, : idx] @ np.diag(vec) @ v[: idx, :]

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [3]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [10]:
def print_result(it, tol, var, var_hat):
    print('Iter: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAPE: {:.6}'.format(compute_mape(var, var_hat)))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(var, var_hat)))
    print()

How to create $\boldsymbol{\Psi}_{0},\boldsymbol{\Psi}_{1},\ldots,\boldsymbol{\Psi}_{d}$?

In [4]:
from scipy import sparse
from scipy.sparse.linalg import spsolve as spsolve

def generate_Psi(dim_time, time_lags):
    Psis = []
    max_lag = np.max(time_lags)
    for i in range(len(time_lags) + 1):
        row = np.arange(0, dim_time - max_lag)
        if i == 0:
            col = np.arange(0, dim_time - max_lag) + max_lag
        else:
            col = np.arange(0, dim_time - max_lag) + max_lag - time_lags[i - 1]
        data = np.ones(dim_time - max_lag)
        Psi = sparse.coo_matrix((data, (row, col)), shape = (dim_time - max_lag, dim_time))
        Psis.append(Psi)
    return Psis

In [5]:
import numpy as np

# Example
dim_time = 5
time_lags = np.array([1, 3])
Psis = generate_Psi(dim_time, time_lags)
print('Psi_0:')
print(Psis[0].toarray())
print()
print('Psi_1:')
print(Psis[1].toarray())
print()
print('Psi_2:')
print(Psis[2].toarray())
print()

Psi_0:
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Psi_1:
[[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]

Psi_2:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]



The main idea behind LATC-imputer is to approximate partially observed data with both low-rank structure and time series dynamics. The following `latc` kernel includes some necessary inputs:

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>dense_tensor</code>:</b> <font color="black">This is an input which has the ground truth for validation. If this input is not available, you could use <code>dense_tensor = sparse_tensor.copy()</code> instead.</font></li>
<li><b><code>sparse_tensor</code>:</b> <font color="black">This is a partially observed tensor which has many missing entries.</font></li>
<li><b><code>time_lags</code>:</b> <font color="black">Time lags, e.g., <code>time_lags = np.array([1, 2, 3])</code>. </font></li>
<li><b><code>alpha</code>:</b> <font color="black">Weights for tensors' nuclear norm, e.g., <code>alpha = np.ones(3) / 3</code>. </font></li>
<li><b><code>rho</code>:</b> <font color="black">Learning rate for ADMM, e.g., <code>rho = 0.0005</code>. </font></li>
<li><b><code>lambda0</code>:</b> <font color="black">Weight for time series regressor, e.g., <code>lambda0 = 5 * rho</code></font></li>
<li><b><code>theta</code>:</b> <font color="black">Integer-wise truncation for truncated nuclear norm, e.g., <code>theta = 5</code></font></li>
<li><b><code>epsilon</code>:</b> <font color="black">Stop criteria, e.g., <code>epsilon = 0.0001</code>. </font></li>
<li><b><code>maxiter</code>:</b> <font color="black">Maximum iteration to stop algorithm, e.g., <code>maxiter = 100</code>. </font></li>
</ul>
</div>

In [62]:
def latc(dense_tensor, sparse_tensor, time_lags, alpha, rho0, lambda0, theta, epsilon, maxiter = 100, K = 3):
    """Low-Rank Autoregressive Tensor Completion (LATC)"""
    
    dim = np.array(sparse_tensor.shape)
    dim_time = np.int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    dense_test = dense_tensor[pos_test]
    del dense_tensor
    
    T = np.zeros(dim)
    Z_tensor = sparse_tensor.copy()
    Z = sparse_mat.copy()
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim_time, time_lags)
    iden = sparse.coo_matrix((np.ones(dim_time), (np.arange(0, dim_time), np.arange(0, dim_time))), 
                             shape = (dim_time, dim_time))
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    while True:
        temp = []
        for m in range(dim[0]):
            Psis0 = Psis.copy()
            for i in range(d):
                Psis0[i + 1] = A[m, i] * Psis[i + 1]
            B = Psis0[0] - sum(Psis0[1 :])
            temp.append(B.T @ B)
        for k in range(K):
            rho = min(rho * 1.05, 1e5)
            tensor_hat = np.zeros(dim)
            for p in range(len(dim)):
                tensor_hat += alpha[p] * mat2ten(svt_tnn(ten2mat(Z_tensor - T / rho, p), 
                                                         alpha[p] / rho, theta), dim, p)
            temp0 = rho / lambda0 * ten2mat(tensor_hat + T / rho, 0)
            mat = np.zeros((dim[0], dim_time))
            for m in range(dim[0]):
                mat[m, :] = spsolve(temp[m] + rho * iden / lambda0, temp0[m, :])
            Z[pos_missing] = mat[pos_missing]
            Z_tensor = mat2ten(Z, dim, 0)
            T = T + rho * (tensor_hat - Z_tensor)
        for m in range(dim[0]):
            A[m, :] = np.linalg.lstsq(Z[m, ind].T, Z[m, max_lag :], rcond = None)[0]
        mat_hat = ten2mat(tensor_hat, 0)
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 200 == 0:
            print_result(it, tol, dense_test, tensor_hat[pos_test])
        if (tol < epsilon) or (it >= maxiter):
            break
    print_result(it, tol, dense_test, tensor_hat[pos_test])
    
    return tensor_hat

> We use `spslove` of `scipy.sparse.linalg` for updating $\boldsymbol{Z}$ because computing the inverse of a large matrix directly is computationally expensive.

### Guangzhou urban traffic speed data set

In [8]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-4
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 32
Tolerance: 9.78464e-05
Imputation MAPE: 0.0778937
Imputation RMSE: 3.28208

Running time: 392 seconds

0.1
10
Total iteration: 33
Tolerance: 9.80482e-05
Imputation MAPE: 0.0746597
Imputation RMSE: 3.17609

Running time: 421 seconds

0.1
15
Total iteration: 34
Tolerance: 9.31732e-05
Imputation MAPE: 0.0725803
Imputation RMSE: 3.10116

Running time: 424 seconds

0.1
20
Total iteration: 34
Tolerance: 9.77646e-05
Imputation MAPE: 0.0710329
Imputation RMSE: 3.04061

Running time: 440 seconds

0.1
25
Total iteration: 35
Tolerance: 9.19364e-05
Imputation MAPE: 0.0700714
Imputation RMSE: 3.00703

Running time: 436 seconds

0.1
30
Total iteration: 36
Tolerance: 9.43022e-05
Imputation MAPE: 0.0694218
Imputation RMSE: 2.98903

Running time: 440 seconds

0.2
5
Total iteration: 32
Tolerance: 8.98847e-05
Imputation MAPE: 0.0774126
Imputation RMSE: 3.26187

Running time: 393 seconds

0.2
10
Total iteration: 33
Tolerance: 9.17651e-05
Imputation MAPE: 0.0742114
Imputation RMSE

In [10]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-4
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 40
Tolerance: 8.79179e-05
Imputation MAPE: 0.0919092
Imputation RMSE: 3.85227

Running time: 557 seconds

0.1
10
Total iteration: 41
Tolerance: 8.87006e-05
Imputation MAPE: 0.087165
Imputation RMSE: 3.69982

Running time: 572 seconds

0.1
15
Total iteration: 41
Tolerance: 9.90424e-05
Imputation MAPE: 0.0848795
Imputation RMSE: 3.62318

Running time: 505 seconds

0.1
20
Total iteration: 42
Tolerance: 9.39873e-05
Imputation MAPE: 0.0838869
Imputation RMSE: 3.59428

Running time: 515 seconds

0.1
25
Total iteration: 43
Tolerance: 8.85303e-05
Imputation MAPE: 0.0838643
Imputation RMSE: 3.6104

Running time: 531 seconds

0.1
30
Total iteration: 43
Tolerance: 9.23608e-05
Imputation MAPE: 0.0848146
Imputation RMSE: 3.67379

Running time: 528 seconds

0.2
5
Total iteration: 39
Tolerance: 9.68902e-05
Imputation MAPE: 0.0916137
Imputation RMSE: 3.83989

Running time: 477 seconds

0.2
10
Total iteration: 40
Tolerance: 9.891e-05
Imputation MAPE: 0.0868958
Imputation RMSE: 3.

In [12]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.9

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-4
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 45
Tolerance: 9.98036e-05
Imputation MAPE: 0.10323
Imputation RMSE: 4.28563

Running time: 591 seconds

0.1
10
Total iteration: 47
Tolerance: 9.63079e-05
Imputation MAPE: 0.0969764
Imputation RMSE: 4.09184

Running time: 590 seconds

0.1
15
Total iteration: 48
Tolerance: 9.91727e-05
Imputation MAPE: 0.0962514
Imputation RMSE: 4.09518

Running time: 650 seconds

0.1
20
Total iteration: 49
Tolerance: 9.58081e-05
Imputation MAPE: 0.0976944
Imputation RMSE: 4.17873

Running time: 664 seconds

0.1
25
Total iteration: 50
Tolerance: 9.28411e-05
Imputation MAPE: 0.100165
Imputation RMSE: 4.30091

Running time: 698 seconds

0.1
30
Total iteration: 51
Tolerance: 9.14001e-05
Imputation MAPE: 0.103347
Imputation RMSE: 4.44577

Running time: 656 seconds

0.2
5
Total iteration: 45
Tolerance: 9.5872e-05
Imputation MAPE: 0.103094
Imputation RMSE: 4.27922

Running time: 589 seconds

0.2
10
Total iteration: 47
Tolerance: 9.25299e-05
Imputation MAPE: 0.0967791
Imputation RMSE: 4.08

In [13]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 46
Tolerance: 8.91646e-05
Imputation MAPE: 0.0983404
Imputation RMSE: 4.13032

Running time: 625 seconds

0.1
10
Total iteration: 48
Tolerance: 9.04685e-05
Imputation MAPE: 0.0962923
Imputation RMSE: 4.09235

Running time: 623 seconds

0.1
15
Total iteration: 50
Tolerance: 9.53406e-05
Imputation MAPE: 0.0959974
Imputation RMSE: 4.10805

Running time: 628 seconds

0.1
20
Total iteration: 52
Tolerance: 9.15908e-05
Imputation MAPE: 0.0971688
Imputation RMSE: 4.19083

Running time: 637 seconds

0.1
25
Total iteration: 53
Tolerance: 9.37119e-05
Imputation MAPE: 0.0984174
Imputation RMSE: 4.27621

Running time: 647 seconds

0.1
30
Total iteration: 54
Tolerance: 9.03176e-05
Imputation MAPE: 0.100944
Imputation RMSE: 4.4113

Running time: 687 seconds

0.2
5
Total iteration: 46
Tolerance: 8.86537e-05
Imputation MAPE: 0.0983318
Imputation RMSE: 4.13016

Running time: 657 seconds

0.2
10
Total iteration: 48
Tolerance: 9.12826e-05
Imputation MAPE: 0.0962858
Imputation RMSE: 

In [15]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 53
Tolerance: 9.74781e-05
Imputation MAPE: 0.103861
Imputation RMSE: 4.35171

Running time: 656 seconds

0.1
10
Total iteration: 57
Tolerance: 9.14753e-05
Imputation MAPE: 0.10267
Imputation RMSE: 4.38248

Running time: 704 seconds

0.1
15
Total iteration: 59
Tolerance: 9.05293e-05
Imputation MAPE: 0.105635
Imputation RMSE: 4.57365

Running time: 727 seconds

0.1
20
Total iteration: 60
Tolerance: 9.19821e-05
Imputation MAPE: 0.1082
Imputation RMSE: 4.68608

Running time: 760 seconds

0.1
25
Total iteration: 61
Tolerance: 8.74206e-05
Imputation MAPE: 0.112723
Imputation RMSE: 4.89443

Running time: 836 seconds

0.1
30
Total iteration: 61
Tolerance: 9.95027e-05
Imputation MAPE: 0.11653
Imputation RMSE: 5.06063

Running time: 787 seconds

0.2
5
Total iteration: 53
Tolerance: 9.79738e-05
Imputation MAPE: 0.103849
Imputation RMSE: 4.35103

Running time: 735 seconds

0.2
10
Total iteration: 57
Tolerance: 9.26729e-05
Imputation MAPE: 0.102589
Imputation RMSE: 4.37181

R

In [16]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape

dim_time = dim2 * dim3
block_window = 6
vec = np.random.rand(int(dim_time / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2 * dim3], order = 'F')

sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 48
Tolerance: 9.18472e-05
Imputation MAPE: 0.0983145
Imputation RMSE: 4.08154

Running time: 674 seconds

0.1
10
Total iteration: 49
Tolerance: 9.56131e-05
Imputation MAPE: 0.0942251
Imputation RMSE: 3.97736

Running time: 677 seconds

0.1
15
Total iteration: 51
Tolerance: 9.66874e-05
Imputation MAPE: 0.0939921
Imputation RMSE: 3.99146

Running time: 703 seconds

0.1
20
Total iteration: 53
Tolerance: 9.6479e-05
Imputation MAPE: 0.0946619
Imputation RMSE: 4.03051

Running time: 744 seconds

0.1
25
Total iteration: 54
Tolerance: 9.54183e-05
Imputation MAPE: 0.0959173
Imputation RMSE: 4.09523

Running time: 785 seconds

0.1
30
Total iteration: 55
Tolerance: 9.78003e-05
Imputation MAPE: 0.0981138
Imputation RMSE: 4.18907

Running time: 805 seconds

0.2
5
Total iteration: 48
Tolerance: 9.1976e-05
Imputation MAPE: 0.0982945
Imputation RMSE: 4.08065

Running time: 676 seconds

0.2
10
Total iteration: 49
Tolerance: 9.55044e-05
Imputation MAPE: 0.0942118
Imputation RMSE: 

In [7]:
import numpy as np
import time
import scipy.io

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Random Missing (RM)
    dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

    start = time.time()
    time_lags = np.array([1, 2, 3, 4, 5, 6])
    alpha = np.ones(3) / 3
    rho = 1e-4
    lambda0 = 5 * rho
    theta = 20
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Iter: 1
Tolerance: 0.216787
MAPE: 0.776689
RMSE: 33.9594

Iter: 2
Tolerance: 0.589551
MAPE: 0.106382
RMSE: 4.28478

Iter: 3
Tolerance: 0.110702
MAPE: 0.10333
RMSE: 4.31134

Iter: 4
Tolerance: 0.0915188
MAPE: 0.101927
RMSE: 4.26182

Iter: 5
Tolerance: 0.0869835
MAPE: 0.0982031
RMSE: 4.10078

Iter: 6
Tolerance: 0.0821847
MAPE: 0.0952909
RMSE: 3.9985

Iter: 7
Tolerance: 0.0742859
MAPE: 0.0886721
RMSE: 3.72192

Iter: 8
Tolerance: 0.0670987
MAPE: 0.0854295
RMSE: 3.58897

Iter: 9
Tolerance: 0.0585255
MAPE: 0.0816279
RMSE: 3.43623

Iter: 10
Tolerance: 0.050865
MAPE: 0.0790695
RMSE: 3.33551

Iter: 11
Tolerance: 0.043802
MAPE: 0.0767001
RMSE: 3.24474

Iter: 12
Tolerance: 0.0367711
MAPE: 0.0748736
RMSE: 3.17449

Iter: 13
Tolerance: 0.0307641
MAPE: 0.0733221
RMSE: 3.11519

Iter: 14
Tolerance: 0.0245294
MAPE: 0.0719857
RMSE: 3.06455

Iter: 15
Tolerance: 0.0199968
MAPE: 0.0708096
RMSE: 3.01922

Iter: 16
Tolerance: 0.0161653
MAPE: 0.0697849
RMSE: 2.98087

Iter: 17
Tolerance: 0.012

Iter: 36
Tolerance: 0.000930614
MAPE: 0.0921133
RMSE: 3.92396

Iter: 37
Tolerance: 0.000813812
MAPE: 0.0920705
RMSE: 3.92294

Iter: 38
Tolerance: 0.000711236
MAPE: 0.0920332
RMSE: 3.92205

Iter: 39
Tolerance: 0.000621182
MAPE: 0.0920006
RMSE: 3.92128

Iter: 40
Tolerance: 0.000542162
MAPE: 0.0919722
RMSE: 3.92061

Iter: 41
Tolerance: 0.000472871
MAPE: 0.0919474
RMSE: 3.92003

Iter: 42
Tolerance: 0.000412153
MAPE: 0.0919258
RMSE: 3.91952

Iter: 43
Tolerance: 0.000358989
MAPE: 0.091907
RMSE: 3.91908

Iter: 44
Tolerance: 0.000312477
MAPE: 0.0918907
RMSE: 3.91869

Iter: 45
Tolerance: 0.000271818
MAPE: 0.0918765
RMSE: 3.91836

Iter: 46
Tolerance: 0.000236305
MAPE: 0.0918642
RMSE: 3.91807

Iter: 47
Tolerance: 0.000205311
MAPE: 0.0918535
RMSE: 3.91782

Iter: 48
Tolerance: 0.000178283
MAPE: 0.0918443
RMSE: 3.91761

Iter: 49
Tolerance: 0.000154733
MAPE: 0.0918362
RMSE: 3.91742

Iter: 50
Tolerance: 0.000134229
MAPE: 0.0918293
RMSE: 3.91726

Iter: 51
Tolerance: 0.00011639
MAPE: 0.0918232
RMSE: 3.9

In [8]:
import numpy as np
import time
import scipy.io

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Non-random Missing (NM)
    dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

    start = time.time()
    time_lags = np.array([1, 2, 3, 4, 5, 6])
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 1 * rho
    theta = 10
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Iter: 1
Tolerance: 0.395123
MAPE: 0.41976
RMSE: 17.4718

Iter: 2
Tolerance: 0.34382
MAPE: 0.154336
RMSE: 5.79753

Iter: 3
Tolerance: 0.0800823
MAPE: 0.115776
RMSE: 4.67226

Iter: 4
Tolerance: 0.143602
MAPE: 0.113467
RMSE: 4.67869

Iter: 5
Tolerance: 0.121501
MAPE: 0.114768
RMSE: 4.69189

Iter: 6
Tolerance: 0.204865
MAPE: 0.123479
RMSE: 4.85991

Iter: 7
Tolerance: 0.205126
MAPE: 0.119396
RMSE: 4.87485

Iter: 8
Tolerance: 0.199487
MAPE: 0.118235
RMSE: 4.82036

Iter: 9
Tolerance: 0.206353
MAPE: 0.12527
RMSE: 4.8902

Iter: 10
Tolerance: 0.223757
MAPE: 0.129368
RMSE: 5.33476

Iter: 11
Tolerance: 0.220295
MAPE: 0.134498
RMSE: 5.42765

Iter: 12
Tolerance: 0.220094
MAPE: 0.138746
RMSE: 5.69114

Iter: 13
Tolerance: 0.228792
MAPE: 0.141586
RMSE: 5.66785

Iter: 14
Tolerance: 0.201291
MAPE: 0.127236
RMSE: 5.24333

Iter: 15
Tolerance: 0.173277
MAPE: 0.123096
RMSE: 5.01316

Iter: 16
Tolerance: 0.149888
MAPE: 0.11619
RMSE: 4.78789

Iter: 17
Tolerance: 0.12977
MAPE: 0.112758
RMSE: 4

In [9]:
import numpy as np
import time
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape

dim_time = dim2 * dim3
block_window = 6
vec = np.random.rand(int(dim_time / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2 * dim3], order = 'F')

sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6])
alpha = np.ones(3) / 3
rho = 1e-5
lambda0 = 5 * rho
theta = 10
epsilon = 1e-4
tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Iter: 1
Tolerance: 0.328203
MAPE: 0.601253
RMSE: 24.8652

Iter: 2
Tolerance: 0.490734
MAPE: 0.157911
RMSE: 6.02819

Iter: 3
Tolerance: 0.114286
MAPE: 0.128278
RMSE: 5.13411

Iter: 4
Tolerance: 0.0208079
MAPE: 0.12509
RMSE: 5.06611

Iter: 5
Tolerance: 0.0263458
MAPE: 0.126135
RMSE: 5.03979

Iter: 6
Tolerance: 0.112243
MAPE: 0.124122
RMSE: 5.18857

Iter: 7
Tolerance: 0.16648
MAPE: 0.131506
RMSE: 5.4562

Iter: 8
Tolerance: 0.153064
MAPE: 0.143515
RMSE: 6.02619

Iter: 9
Tolerance: 0.219256
MAPE: 0.147358
RMSE: 6.29989

Iter: 10
Tolerance: 0.278774
MAPE: 0.146976
RMSE: 6.32651

Iter: 11
Tolerance: 0.236985
MAPE: 0.13872
RMSE: 5.82743

Iter: 12
Tolerance: 0.209948
MAPE: 0.139651
RMSE: 5.86687

Iter: 13
Tolerance: 0.220177
MAPE: 0.137124
RMSE: 5.70384

Iter: 14
Tolerance: 0.208337
MAPE: 0.126408
RMSE: 5.22177

Iter: 15
Tolerance: 0.184261
MAPE: 0.122494
RMSE: 5.03351

Iter: 16
Tolerance: 0.155835
MAPE: 0.114298
RMSE: 4.7119

Iter: 17
Tolerance: 0.134128
MAPE: 0.107932
RMSE: 4.44373

Iter: 18


### Hangzhou metro passenger flow data set

In [8]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 42
Tolerance: 9.28827e-05
Imputation MAPE: 0.188196
Imputation RMSE: 25.5156

Running time: 58 seconds

0.1
10
Total iteration: 44
Tolerance: 8.82417e-05
Imputation MAPE: 0.189751
Imputation RMSE: 25.5395

Running time: 65 seconds

0.1
15
Total iteration: 47
Tolerance: 8.92199e-05
Imputation MAPE: 0.19247
Imputation RMSE: 26.3031

Running time: 63 seconds

0.1
20
Total iteration: 48
Tolerance: 9.55468e-05
Imputation MAPE: 0.200957
Imputation RMSE: 28.1413

Running time: 67 seconds

0.2
5
Total iteration: 42
Tolerance: 9.22564e-05
Imputation MAPE: 0.188136
Imputation RMSE: 25.4745

Running time: 57 seconds

0.2
10
Total iteration: 44
Tolerance: 8.8493e-05
Imputation MAPE: 0.189663
Imputation RMSE: 25.4089

Running time: 56 seconds

0.2
15
Total iteration: 47
Tolerance: 8.72728e-05
Imputation MAPE: 0.192303
Imputation RMSE: 25.9186

Running time: 65 seconds

0.2
20
Total iteration: 48
Tolerance: 9.42147e-05
Imputation MAPE: 0.200282
Imputation RMSE: 27.647

Running

In [9]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 50
Tolerance: 9.13175e-05
Imputation MAPE: 0.203768
Imputation RMSE: 28.5683

Running time: 71 seconds

0.1
10
Total iteration: 52
Tolerance: 9.71072e-05
Imputation MAPE: 0.201502
Imputation RMSE: 28.8334

Running time: 72 seconds

0.1
15
Total iteration: 54
Tolerance: 9.43154e-05
Imputation MAPE: 0.208074
Imputation RMSE: 29.6451

Running time: 83 seconds

0.1
20
Total iteration: 55
Tolerance: 9.59401e-05
Imputation MAPE: 0.224546
Imputation RMSE: 31.1883

Running time: 76 seconds

0.2
5
Total iteration: 50
Tolerance: 9.22674e-05
Imputation MAPE: 0.203945
Imputation RMSE: 28.5279

Running time: 76 seconds

0.2
10
Total iteration: 52
Tolerance: 9.96029e-05
Imputation MAPE: 0.201446
Imputation RMSE: 28.4431

Running time: 72 seconds

0.2
15
Total iteration: 54
Tolerance: 8.98637e-05
Imputation MAPE: 0.208712
Imputation RMSE: 29.5614

Running time: 75 seconds

0.2
20
Total iteration: 55
Tolerance: 9.43032e-05
Imputation MAPE: 0.222302
Imputation RMSE: 31.0886

Runn

In [10]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.9

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 58
Tolerance: 8.62476e-05
Imputation MAPE: 0.253496
Imputation RMSE: 36.194

Running time: 73 seconds

0.1
10
Total iteration: 60
Tolerance: 9.38225e-05
Imputation MAPE: 0.247054
Imputation RMSE: 34.814

Running time: 75 seconds

0.1
15
Total iteration: 62
Tolerance: 8.92342e-05
Imputation MAPE: 0.268914
Imputation RMSE: 36.5471

Running time: 79 seconds

0.1
20
Total iteration: 64
Tolerance: 8.65013e-05
Imputation MAPE: 0.296798
Imputation RMSE: 37.829

Running time: 79 seconds

0.2
5
Total iteration: 58
Tolerance: 8.61807e-05
Imputation MAPE: 0.253113
Imputation RMSE: 36.2331

Running time: 71 seconds

0.2
10
Total iteration: 60
Tolerance: 9.33359e-05
Imputation MAPE: 0.247578
Imputation RMSE: 34.6763

Running time: 74 seconds

0.2
15
Total iteration: 62
Tolerance: 8.76038e-05
Imputation MAPE: 0.267456
Imputation RMSE: 36.7431

Running time: 77 seconds

0.2
20
Total iteration: 63
Tolerance: 9.81735e-05
Imputation MAPE: 0.293918
Imputation RMSE: 37.8606

Running

In [11]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 44
Tolerance: 8.71246e-05
Imputation MAPE: 0.199306
Imputation RMSE: 47.3845

Running time: 62 seconds

0.1
10
Total iteration: 47
Tolerance: 9.51003e-05
Imputation MAPE: 0.197376
Imputation RMSE: 50.1533

Running time: 64 seconds

0.1
15
Total iteration: 48
Tolerance: 9.31785e-05
Imputation MAPE: 0.197134
Imputation RMSE: 50.3337

Running time: 62 seconds

0.1
20
Total iteration: 49
Tolerance: 9.15123e-05
Imputation MAPE: 0.206232
Imputation RMSE: 52.2789

Running time: 65 seconds

0.2
5
Total iteration: 44
Tolerance: 8.76025e-05
Imputation MAPE: 0.199684
Imputation RMSE: 47.4428

Running time: 56 seconds

0.2
10
Total iteration: 47
Tolerance: 8.80166e-05
Imputation MAPE: 0.19791
Imputation RMSE: 50.4234

Running time: 60 seconds

0.2
15
Total iteration: 48
Tolerance: 9.13788e-05
Imputation MAPE: 0.197805
Imputation RMSE: 49.5722

Running time: 59 seconds

0.2
20
Total iteration: 49
Tolerance: 9.51457e-05
Imputation MAPE: 0.206722
Imputation RMSE: 52.3443

Runni

In [12]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 51
Tolerance: 9.60428e-05
Imputation MAPE: 0.242815
Imputation RMSE: 47.5299

Running time: 63 seconds

0.1
10
Total iteration: 54
Tolerance: 8.7461e-05
Imputation MAPE: 0.240149
Imputation RMSE: 50.9818

Running time: 67 seconds

0.1
15
Total iteration: 55
Tolerance: 9.54954e-05
Imputation MAPE: 0.257818
Imputation RMSE: 52.748

Running time: 68 seconds

0.1
20
Total iteration: 55
Tolerance: 9.92918e-05
Imputation MAPE: 0.284349
Imputation RMSE: 54.2794

Running time: 68 seconds

0.2
5
Total iteration: 51
Tolerance: 9.79881e-05
Imputation MAPE: 0.243001
Imputation RMSE: 47.2979

Running time: 63 seconds

0.2
10
Total iteration: 54
Tolerance: 8.90281e-05
Imputation MAPE: 0.237706
Imputation RMSE: 50.8872

Running time: 67 seconds

0.2
15
Total iteration: 55
Tolerance: 9.36855e-05
Imputation MAPE: 0.254516
Imputation RMSE: 52.4596

Running time: 68 seconds

0.2
20
Total iteration: 55
Tolerance: 9.63348e-05
Imputation MAPE: 0.279654
Imputation RMSE: 53.6904

Runnin

In [13]:
import time
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape

dim_time = dim2 * dim3
block_window = 6
vec = np.random.rand(int(dim_time / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2 * dim3], order = 'F')

sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 42
Tolerance: 9.72447e-05
Imputation MAPE: 0.216939
Imputation RMSE: 28.8341

Running time: 53 seconds

0.1
10
Total iteration: 45
Tolerance: 9.85845e-05
Imputation MAPE: 0.218723
Imputation RMSE: 28.7348

Running time: 58 seconds

0.1
15
Total iteration: 48
Tolerance: 9.98682e-05
Imputation MAPE: 0.217535
Imputation RMSE: 29.8191

Running time: 66 seconds

0.1
20
Total iteration: 50
Tolerance: 8.92749e-05
Imputation MAPE: 0.228724
Imputation RMSE: 32.0615

Running time: 63 seconds

0.2
5
Total iteration: 42
Tolerance: 9.7935e-05
Imputation MAPE: 0.216923
Imputation RMSE: 28.8116

Running time: 61 seconds

0.2
10
Total iteration: 45
Tolerance: 9.92066e-05
Imputation MAPE: 0.219212
Imputation RMSE: 28.7026

Running time: 62 seconds

0.2
15
Total iteration: 49
Tolerance: 8.74318e-05
Imputation MAPE: 0.217434
Imputation RMSE: 29.6215

Running time: 66 seconds

0.2
20
Total iteration: 50
Tolerance: 8.76989e-05
Imputation MAPE: 0.226072
Imputation RMSE: 31.6499

Runni

In [10]:
import numpy as np
import time
import scipy.io

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Random Missing (RM)
    dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

    start = time.time()
    time_lags = np.array([1, 2, 3, 4, 5, 6])
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 1 * rho
    theta = 10
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Iter: 1
Tolerance: 0.54603
MAPE: 0.383973
RMSE: 64.6995

Iter: 2
Tolerance: 0.271162
MAPE: 0.295848
RMSE: 40.7109

Iter: 3
Tolerance: 0.140841
MAPE: 0.237724
RMSE: 34.3527

Iter: 4
Tolerance: 0.114311
MAPE: 0.26236
RMSE: 34.6816

Iter: 5
Tolerance: 0.166582
MAPE: 0.247924
RMSE: 32.8197

Iter: 6
Tolerance: 0.199337
MAPE: 0.306918
RMSE: 36.7673

Iter: 7
Tolerance: 0.196401
MAPE: 0.253883
RMSE: 31.9747

Iter: 8
Tolerance: 0.154357
MAPE: 0.256985
RMSE: 32.0374

Iter: 9
Tolerance: 0.141914
MAPE: 0.249341
RMSE: 30.9281

Iter: 10
Tolerance: 0.13223
MAPE: 0.237222
RMSE: 29.7254

Iter: 11
Tolerance: 0.122951
MAPE: 0.237057
RMSE: 28.6582

Iter: 12
Tolerance: 0.0991007
MAPE: 0.223439
RMSE: 27.8302

Iter: 13
Tolerance: 0.0958206
MAPE: 0.2207
RMSE: 27.5568

Iter: 14
Tolerance: 0.0868176
MAPE: 0.21519
RMSE: 26.7852

Iter: 15
Tolerance: 0.0826815
MAPE: 0.211888
RMSE: 26.6206

Iter: 16
Tolerance: 0.073443
MAPE: 0.204493
RMSE: 26.1331

Iter: 17
Tolerance: 0.0579859
MAPE: 0.201099
RMS

Iter: 37
Tolerance: 0.00247585
MAPE: 0.244906
RMSE: 34.4592

Iter: 38
Tolerance: 0.0021462
MAPE: 0.244707
RMSE: 34.4553

Iter: 39
Tolerance: 0.00186476
MAPE: 0.244529
RMSE: 34.4522

Iter: 40
Tolerance: 0.00161412
MAPE: 0.244368
RMSE: 34.4497

Iter: 41
Tolerance: 0.00139833
MAPE: 0.244223
RMSE: 34.4477

Iter: 42
Tolerance: 0.00121433
MAPE: 0.244093
RMSE: 34.4461

Iter: 43
Tolerance: 0.00105689
MAPE: 0.243975
RMSE: 34.4449

Iter: 44
Tolerance: 0.000919212
MAPE: 0.243864
RMSE: 34.4438

Iter: 45
Tolerance: 0.000798146
MAPE: 0.243752
RMSE: 34.443

Iter: 46
Tolerance: 0.000691627
MAPE: 0.243647
RMSE: 34.4424

Iter: 47
Tolerance: 0.000599037
MAPE: 0.243557
RMSE: 34.4418

Iter: 48
Tolerance: 0.000519916
MAPE: 0.243478
RMSE: 34.4414

Iter: 49
Tolerance: 0.000451973
MAPE: 0.243412
RMSE: 34.441

Iter: 50
Tolerance: 0.000393479
MAPE: 0.243359
RMSE: 34.4407

Iter: 51
Tolerance: 0.000340807
MAPE: 0.243317
RMSE: 34.4405

Iter: 52
Tolerance: 0.000292573
MAPE: 0.243282
RMSE: 34.4403

Iter: 53
Tolerance

In [11]:
import numpy as np
import time
import scipy.io

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Non-random Missing (NM)
    dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

    start = time.time()
    time_lags = np.array([1, 2, 3, 4, 5, 6])
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 1 * rho
    theta = 5
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Iter: 1
Tolerance: 0.395519
MAPE: 0.597662
RMSE: 161.815

Iter: 2
Tolerance: 0.417849
MAPE: 0.269355
RMSE: 91.1584

Iter: 3
Tolerance: 0.168727
MAPE: 0.221407
RMSE: 68.9444

Iter: 4
Tolerance: 0.139779
MAPE: 0.226639
RMSE: 60.7735

Iter: 5
Tolerance: 0.144749
MAPE: 0.241207
RMSE: 58.0078

Iter: 6
Tolerance: 0.179011
MAPE: 0.256376
RMSE: 57.3385

Iter: 7
Tolerance: 0.18734
MAPE: 0.271309
RMSE: 58.0295

Iter: 8
Tolerance: 0.162207
MAPE: 0.249902
RMSE: 57.1268

Iter: 9
Tolerance: 0.161508
MAPE: 0.240294
RMSE: 56.6131

Iter: 10
Tolerance: 0.136672
MAPE: 0.235838
RMSE: 56.0849

Iter: 11
Tolerance: 0.129116
MAPE: 0.225676
RMSE: 55.2512

Iter: 12
Tolerance: 0.104323
MAPE: 0.223866
RMSE: 55.0726

Iter: 13
Tolerance: 0.0945764
MAPE: 0.216113
RMSE: 54.2356

Iter: 14
Tolerance: 0.0815312
MAPE: 0.21504
RMSE: 53.6885

Iter: 15
Tolerance: 0.064167
MAPE: 0.211738
RMSE: 52.9244

Iter: 16
Tolerance: 0.0525267
MAPE: 0.209959
RMSE: 52.1471

Iter: 17
Tolerance: 0.0398846
MAPE: 0.209042


In [12]:
import numpy as np
import time
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape

dim_time = dim2 * dim3
block_window = 6
vec = np.random.rand(int(dim_time / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2 * dim3], order = 'F')

sparse_tensor = mat2ten(ten2mat(dense_tensor, 0) * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6])
alpha = np.ones(3) / 3
rho = 1e-5
lambda0 = 1 * rho
theta = 10
epsilon = 1e-4
tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Iter: 1
Tolerance: 0.400763
MAPE: 0.655301
RMSE: 118.626

Iter: 2
Tolerance: 0.440321
MAPE: 0.319361
RMSE: 44.2598

Iter: 3
Tolerance: 0.176614
MAPE: 0.268209
RMSE: 37.2718

Iter: 4
Tolerance: 0.190615
MAPE: 0.259467
RMSE: 33.9339

Iter: 5
Tolerance: 0.166321
MAPE: 0.251865
RMSE: 33.5717

Iter: 6
Tolerance: 0.173695
MAPE: 0.312447
RMSE: 38.9513

Iter: 7
Tolerance: 0.195685
MAPE: 0.309044
RMSE: 38.2189

Iter: 8
Tolerance: 0.198703
MAPE: 0.282938
RMSE: 35.2562

Iter: 9
Tolerance: 0.156732
MAPE: 0.269298
RMSE: 33.779

Iter: 10
Tolerance: 0.137372
MAPE: 0.256122
RMSE: 32.2365

Iter: 11
Tolerance: 0.145039
MAPE: 0.25616
RMSE: 31.9278

Iter: 12
Tolerance: 0.12778
MAPE: 0.246421
RMSE: 30.4501

Iter: 13
Tolerance: 0.113447
MAPE: 0.24653
RMSE: 29.9119

Iter: 14
Tolerance: 0.106221
MAPE: 0.246521
RMSE: 29.4578

Iter: 15
Tolerance: 0.0918672
MAPE: 0.241862
RMSE: 29.2343

Iter: 16
Tolerance: 0.0828478
MAPE: 0.239431
RMSE: 29.0114

Iter: 17
Tolerance: 0.0705952
MAPE: 0.238606
RMSE: 28.9161

Iter: 1

### Seattle freeway traffic speed data set

In [14]:
import numpy as np
import pandas as pd
import time
import scipy.io

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Random missing (RM)
    dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0).values
    dense_tensor = dense_mat.reshape([dense_mat.shape[0], 28, 288]).transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

    start = time.time()
    time_lags = np.arange(1, 7)
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 10 * rho
    theta = 25
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Total iteration: 45
Tolerance: 8.77294e-05
Imputation MAPE: 0.0489623
Imputation RMSE: 3.15582

Running time: 697 seconds

Missing rate = 0.7
Total iteration: 52
Tolerance: 9.87315e-05
Imputation MAPE: 0.0596453
Imputation RMSE: 3.70916

Running time: 819 seconds

Missing rate = 0.9
Total iteration: 62
Tolerance: 9.31205e-05
Imputation MAPE: 0.0747038
Imputation RMSE: 4.50685

Running time: 948 seconds



In [7]:
import numpy as np
import pandas as pd
import time
import scipy.io

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Non-random Missing (NM)
    dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0).values
    dense_tensor = dense_mat.reshape([dense_mat.shape[0], 28, 288]).transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

    start = time.time()
    time_lags = np.arange(1, 7)
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 10 * rho
    if r == 0.3:
        theta = 25
    elif r == 0.7:
        theta = 10
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Total iteration: 46
Tolerance: 9.90209e-05
Imputation MAPE: 0.0710509
Imputation RMSE: 4.32765

Running time: 718 seconds

Missing rate = 0.7
Total iteration: 54
Tolerance: 9.08842e-05
Imputation MAPE: 0.0946159
Imputation RMSE: 5.42155

Running time: 796 seconds



In [8]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0).values
dense_tensor = dense_mat.reshape([dense_mat.shape[0], 28, 288]).transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
block_window = 12
vec = np.random.rand(int(dim2 * dim3 / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2 * dim3], order = 'F')
sparse_tensor = mat2ten(dense_mat * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

start = time.time()
time_lags = np.arange(1, 7)
alpha = np.ones(3) / 3
rho = 1e-5
lambda0 = 10 * rho
theta = 10
epsilon = 1e-4
tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Total iteration: 48
Tolerance: 8.98277e-05
Imputation MAPE: 0.094364
Imputation RMSE: 5.35739

Running time: 704 seconds



### Portland highway traffic volume data set

In [16]:
import numpy as np
import pandas as pd
import time
import scipy.io

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    # Random Missing (RM)
    dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    np.random.seed(1000)
    sparse_tensor = mat2ten(dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate), dim, 0)

    start = time.time()
    time_lags = np.array([1, 2, 3, 4])
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 1 * rho
    theta = 20
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Iter: 1
Tolerance: 0.426691
MAPE: 0.538394
RMSE: 81.5098

Iter: 2
Tolerance: 0.366846
MAPE: 0.284658
RMSE: 25.7461

Iter: 3
Tolerance: 0.155325
MAPE: 0.269354
RMSE: 23.4096

Iter: 4
Tolerance: 0.121972
MAPE: 0.250068
RMSE: 21.8084

Iter: 5
Tolerance: 0.110874
MAPE: 0.240239
RMSE: 21.4105

Iter: 6
Tolerance: 0.0974799
MAPE: 0.227197
RMSE: 20.4566

Iter: 7
Tolerance: 0.0902777
MAPE: 0.219335
RMSE: 19.7096

Iter: 8
Tolerance: 0.0805582
MAPE: 0.214195
RMSE: 19.5049

Iter: 9
Tolerance: 0.0770269
MAPE: 0.210902
RMSE: 19.3672

Iter: 10
Tolerance: 0.0709719
MAPE: 0.204247
RMSE: 18.7964

Iter: 11
Tolerance: 0.0641586
MAPE: 0.197479
RMSE: 18.2638

Iter: 12
Tolerance: 0.0541934
MAPE: 0.195244
RMSE: 18.1517

Iter: 13
Tolerance: 0.0465736
MAPE: 0.192531
RMSE: 17.9867

Iter: 14
Tolerance: 0.0385302
MAPE: 0.191057
RMSE: 17.8825

Iter: 15
Tolerance: 0.0311717
MAPE: 0.188845
RMSE: 17.7586

Iter: 16
Tolerance: 0.0246696
MAPE: 0.186487
RMSE: 17.6181

Iter: 17
Tolerance: 0.0203487
MAPE:

Iter: 31
Tolerance: 0.00630344
MAPE: 0.231023
RMSE: 22.8867

Iter: 32
Tolerance: 0.00560083
MAPE: 0.23057
RMSE: 22.8578

Iter: 33
Tolerance: 0.00495541
MAPE: 0.230241
RMSE: 22.8364

Iter: 34
Tolerance: 0.00436882
MAPE: 0.230024
RMSE: 22.8207

Iter: 35
Tolerance: 0.003844
MAPE: 0.229899
RMSE: 22.8094

Iter: 36
Tolerance: 0.0033742
MAPE: 0.229848
RMSE: 22.8013

Iter: 37
Tolerance: 0.00295628
MAPE: 0.229857
RMSE: 22.7956

Iter: 38
Tolerance: 0.00258793
MAPE: 0.229912
RMSE: 22.7917

Iter: 39
Tolerance: 0.00226349
MAPE: 0.230001
RMSE: 22.789

Iter: 40
Tolerance: 0.00197768
MAPE: 0.230112
RMSE: 22.7873

Iter: 41
Tolerance: 0.00172629
MAPE: 0.230239
RMSE: 22.7862

Iter: 42
Tolerance: 0.00150567
MAPE: 0.230375
RMSE: 22.7856

Iter: 43
Tolerance: 0.00131224
MAPE: 0.230513
RMSE: 22.7853

Iter: 44
Tolerance: 0.00114264
MAPE: 0.230649
RMSE: 22.7852

Iter: 45
Tolerance: 0.000994296
MAPE: 0.230781
RMSE: 22.7852

Iter: 46
Tolerance: 0.000864625
MAPE: 0.230905
RMSE: 22.7853

Iter: 47
Tolerance: 0.00075

In [22]:
import numpy as np
import pandas as pd
import time
import scipy.io

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    # Non-random Missing (NM)
    dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim[2]) + 0.5 - missing_rate)[:, None, :]

    start = time.time()
    time_lags = np.array([1, 2, 3, 4])
    alpha = np.ones(3) / 3
    rho = 1e-5
    lambda0 = 1 * rho
    theta = 5
    epsilon = 1e-4
    tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Missing rate = 0.3
Iter: 1
Tolerance: 0.320105
MAPE: 0.619532
RMSE: 114.606

Iter: 2
Tolerance: 0.413555
MAPE: 0.279051
RMSE: 30.5509

Iter: 3
Tolerance: 0.117901
MAPE: 0.259152
RMSE: 25.7152

Iter: 4
Tolerance: 0.0903665
MAPE: 0.259956
RMSE: 25.2961

Iter: 5
Tolerance: 0.0627115
MAPE: 0.251993
RMSE: 24.5702

Iter: 6
Tolerance: 0.0589839
MAPE: 0.251455
RMSE: 24.4283

Iter: 7
Tolerance: 0.0431803
MAPE: 0.24662
RMSE: 24.0508

Iter: 8
Tolerance: 0.0366578
MAPE: 0.244003
RMSE: 23.8455

Iter: 9
Tolerance: 0.0311811
MAPE: 0.240604
RMSE: 23.5249

Iter: 10
Tolerance: 0.0274583
MAPE: 0.237612
RMSE: 23.232

Iter: 11
Tolerance: 0.0165415
MAPE: 0.234529
RMSE: 22.9637

Iter: 12
Tolerance: 0.0151128
MAPE: 0.231631
RMSE: 22.6834

Iter: 13
Tolerance: 0.0130727
MAPE: 0.228582
RMSE: 22.4054

Iter: 14
Tolerance: 0.00943626
MAPE: 0.225529
RMSE: 22.1451

Iter: 15
Tolerance: 0.00679548
MAPE: 0.22244
RMSE: 21.8757

Iter: 16
Tolerance: 0.0043657
MAPE: 0.219376
RMSE: 21.6166

Iter: 17
Tolerance: 0.00317734
MAP

In [23]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
block_window = 4
vec = np.random.rand(int(dim2 / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2], order = 'F')
sparse_tensor = mat2ten(dense_mat * np.round(vec + 0.5 - missing_rate)[None, :], dim, 0)

start = time.time()
time_lags = np.array([1, 2, 3, 4])
alpha = np.ones(3) / 3
rho = 1e-5
lambda0 = 1 * rho
theta = 5
epsilon = 1e-4
tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Iter: 1
Tolerance: 0.221849
MAPE: 0.775239
RMSE: 157.482

Iter: 2
Tolerance: 0.389708
MAPE: 0.404607
RMSE: 80.4288

Iter: 3
Tolerance: 0.181334
MAPE: 0.306371
RMSE: 50.9144

Iter: 4
Tolerance: 0.117606
MAPE: 0.277314
RMSE: 36.5207

Iter: 5
Tolerance: 0.0803419
MAPE: 0.257466
RMSE: 28.1712

Iter: 6
Tolerance: 0.0572728
MAPE: 0.253112
RMSE: 24.8783

Iter: 7
Tolerance: 0.0423767
MAPE: 0.248087
RMSE: 23.9322

Iter: 8
Tolerance: 0.0334939
MAPE: 0.247652
RMSE: 23.5497

Iter: 9
Tolerance: 0.0275865
MAPE: 0.246314
RMSE: 23.4969

Iter: 10
Tolerance: 0.0249986
MAPE: 0.245633
RMSE: 23.4359

Iter: 11
Tolerance: 0.0165848
MAPE: 0.244498
RMSE: 23.4424

Iter: 12
Tolerance: 0.0111048
MAPE: 0.243377
RMSE: 23.4243

Iter: 13
Tolerance: 0.0113552
MAPE: 0.242596
RMSE: 23.4388

Iter: 14
Tolerance: 0.0108989
MAPE: 0.241983
RMSE: 23.4506

Iter: 15
Tolerance: 0.00907551
MAPE: 0.24171
RMSE: 23.4677

Iter: 16
Tolerance: 0.00798179
MAPE: 0.241551
RMSE: 23.4816

Iter: 17
Tolerance: 0.00669739
MAPE: 0.241595
RMSE: 

In [66]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0).values
dense_tensor = dense_mat.reshape([dense_mat.shape[0], 28, 288]).transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 44
Tolerance: 8.94528e-05
Imputation MAPE: 0.0796457
Imputation RMSE: 4.6528

Running time: 659 seconds

0.1
10
Total iteration: 45
Tolerance: 9.53995e-05
Imputation MAPE: 0.0751242
Imputation RMSE: 4.48422

Running time: 666 seconds

0.1
15
Total iteration: 47
Tolerance: 9.11372e-05
Imputation MAPE: 0.073125
Imputation RMSE: 4.39517

Running time: 698 seconds

0.1
20
Total iteration: 47
Tolerance: 9.31234e-05
Imputation MAPE: 0.0717918
Imputation RMSE: 4.35902

Running time: 693 seconds

0.1
25
Total iteration: 47
Tolerance: 9.47087e-05
Imputation MAPE: 0.0708676
Imputation RMSE: 4.35017

Running time: 690 seconds

0.2
5
Total iteration: 44
Tolerance: 8.93657e-05
Imputation MAPE: 0.0796539
Imputation RMSE: 4.65314

Running time: 646 seconds

0.2
10
Total iteration: 45
Tolerance: 9.62575e-05
Imputation MAPE: 0.0751295
Imputation RMSE: 4.48409

Running time: 663 seconds

0.2
15
Total iteration: 47
Tolerance: 9.14917e-05
Imputation MAPE: 0.0731577
Imputation RMSE: 

In [67]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0).values
dense_tensor = dense_mat.reshape([dense_mat.shape[0], 28, 288]).transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 52
Tolerance: 8.98251e-05
Imputation MAPE: 0.0990359
Imputation RMSE: 5.58469

Running time: 809 seconds

0.1
10
Total iteration: 54
Tolerance: 9.86017e-05
Imputation MAPE: 0.0941775
Imputation RMSE: 5.42502

Running time: 859 seconds

0.1
15
Total iteration: 56
Tolerance: 9.0966e-05
Imputation MAPE: 0.0950379
Imputation RMSE: 5.53102

Running time: 935 seconds

0.1
20
Total iteration: 57
Tolerance: 9.37997e-05
Imputation MAPE: 0.0949351
Imputation RMSE: 5.55727

Running time: 1055 seconds

0.1
25
Total iteration: 58
Tolerance: 9.1737e-05
Imputation MAPE: 0.0952095
Imputation RMSE: 5.61701

Running time: 946 seconds

0.2
5
Total iteration: 52
Tolerance: 8.9781e-05
Imputation MAPE: 0.0990413
Imputation RMSE: 5.58483

Running time: 857 seconds

0.2
10
Total iteration: 54
Tolerance: 9.96331e-05
Imputation MAPE: 0.0943331
Imputation RMSE: 5.42713

Running time: 890 seconds

0.2
15
Total iteration: 56
Tolerance: 9.1326e-05
Imputation MAPE: 0.0946924
Imputation RMSE: 5

In [61]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0).values
dense_tensor = dense_mat.reshape([dense_mat.shape[0], 28, 288]).transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
block_window = 12
vec = np.random.rand(int(dim2 * dim3 / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2 * dim3], order = 'F')
sparse_tensor = mat2ten(dense_mat * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.arange(1, 7)
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

Total iteration: 45
Tolerance: 9.40962e-05
Imputation MAPE: 0.100147
Imputation RMSE: 5.58299

Running time: 665 seconds

Total iteration: 48
Tolerance: 9.66093e-05
Imputation MAPE: 0.0959433
Imputation RMSE: 5.46633

Running time: 707 seconds

Total iteration: 50
Tolerance: 8.79303e-05
Imputation MAPE: 0.0985432
Imputation RMSE: 5.58353

Running time: 741 seconds

Total iteration: 50
Tolerance: 9.13246e-05
Imputation MAPE: 0.100937
Imputation RMSE: 5.68708

Running time: 735 seconds

Total iteration: 50
Tolerance: 8.72269e-05
Imputation MAPE: 0.0994611
Imputation RMSE: 5.61749

Running time: 735 seconds

Total iteration: 52
Tolerance: 9.79445e-05
Imputation MAPE: 0.0993687
Imputation RMSE: 5.63752

Running time: 766 seconds

Total iteration: 45
Tolerance: 9.39613e-05
Imputation MAPE: 0.10012
Imputation RMSE: 5.58176

Running time: 672 seconds

Total iteration: 48
Tolerance: 9.64832e-05
Imputation MAPE: 0.0958204
Imputation RMSE: 5.46171

Running time: 766 seconds

Total iteration: 50


In [59]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

# Random Missing (RM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
np.random.seed(1000)
sparse_tensor = mat2ten(dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate), dim, 0)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.array([1, 2, 3, 4])
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 56
Tolerance: 8.76659e-05
Imputation MAPE: 0.210986
Imputation RMSE: 19.8511

Running time: 1288 seconds

0.1
10
Total iteration: 56
Tolerance: 9.00153e-05
Imputation MAPE: 0.205165
Imputation RMSE: 19.2119

Running time: 1033 seconds

0.1
15
Total iteration: 56
Tolerance: 9.28019e-05
Imputation MAPE: 0.201297
Imputation RMSE: 19.2179

Running time: 1013 seconds

0.1
20
Total iteration: 56
Tolerance: 9.6778e-05
Imputation MAPE: 0.199432
Imputation RMSE: 19.2013

Running time: 1015 seconds

0.1
25
Total iteration: 57
Tolerance: 8.69177e-05
Imputation MAPE: 0.198674
Imputation RMSE: 19.083

Running time: 1030 seconds

0.1
30
Total iteration: 57
Tolerance: 9.08699e-05
Imputation MAPE: 0.198538
Imputation RMSE: 18.9252

Running time: 1029 seconds

0.2
5
Total iteration: 56
Tolerance: 8.68102e-05
Imputation MAPE: 0.210701
Imputation RMSE: 19.8254

Running time: 1013 seconds

0.2
10
Total iteration: 56
Tolerance: 8.91622e-05
Imputation MAPE: 0.204812
Imputation RMSE: 1

In [60]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.9

# Random Missing (RM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
np.random.seed(1000)
sparse_tensor = mat2ten(dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate), dim, 0)

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.array([1, 2, 3, 4])
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 61
Tolerance: 8.70976e-05
Imputation MAPE: 0.250613
Imputation RMSE: 24.2772

Running time: 1130 seconds

0.1
10
Total iteration: 61
Tolerance: 8.99146e-05
Imputation MAPE: 0.240087
Imputation RMSE: 23.3185

Running time: 1165 seconds

0.1
15
Total iteration: 61
Tolerance: 9.47273e-05
Imputation MAPE: 0.23756
Imputation RMSE: 23.1359

Running time: 1240 seconds

0.1
20
Total iteration: 62
Tolerance: 8.72224e-05
Imputation MAPE: 0.236036
Imputation RMSE: 23.0396

Running time: 1176 seconds

0.1
25
Total iteration: 62
Tolerance: 9.38941e-05
Imputation MAPE: 0.237517
Imputation RMSE: 23.0914

Running time: 1131 seconds

0.1
30
Total iteration: 63
Tolerance: 8.80237e-05
Imputation MAPE: 0.239715
Imputation RMSE: 23.2986

Running time: 1145 seconds

0.2
5
Total iteration: 61
Tolerance: 8.67679e-05
Imputation MAPE: 0.250004
Imputation RMSE: 24.2121

Running time: 1107 seconds

0.2
10
Total iteration: 61
Tolerance: 8.94507e-05
Imputation MAPE: 0.239709
Imputation RMSE: 

In [57]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

# Non-random Missing (NM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim[2]) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.array([1, 2, 3, 4])
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 46
Tolerance: 9.70608e-05
Imputation MAPE: 0.199461
Imputation RMSE: 19.6054

Running time: 903 seconds

0.1
10
Total iteration: 47
Tolerance: 8.9338e-05
Imputation MAPE: 0.195803
Imputation RMSE: 18.978

Running time: 933 seconds

0.1
15
Total iteration: 47
Tolerance: 9.46995e-05
Imputation MAPE: 0.193094
Imputation RMSE: 19.0461

Running time: 973 seconds

0.1
20
Total iteration: 48
Tolerance: 8.60834e-05
Imputation MAPE: 0.19139
Imputation RMSE: 18.9715

Running time: 1085 seconds

0.1
25
Total iteration: 48
Tolerance: 8.96996e-05
Imputation MAPE: 0.18968
Imputation RMSE: 18.9075

Running time: 1095 seconds

0.1
30
Total iteration: 48
Tolerance: 9.3357e-05
Imputation MAPE: 0.189889
Imputation RMSE: 19.1696

Running time: 1054 seconds

0.2
5
Total iteration: 46
Tolerance: 9.45733e-05
Imputation MAPE: 0.199286
Imputation RMSE: 19.6142

Running time: 1039 seconds

0.2
10
Total iteration: 47
Tolerance: 8.61707e-05
Imputation MAPE: 0.195495
Imputation RMSE: 19.0326

In [58]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

# Non-random Missing (NM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
np.random.seed(1000)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim[2]) + 0.5 - missing_rate)[:, None, :]

for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5, 10, 15, 20, 25, 30]:
        start = time.time()
        time_lags = np.array([1, 2, 3, 4])
        alpha = np.ones(3) / 3
        rho = 1e-5
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 56
Tolerance: 8.79369e-05
Imputation MAPE: 0.257048
Imputation RMSE: 32.5063

Running time: 1276 seconds

0.1
10
Total iteration: 56
Tolerance: 9.13703e-05
Imputation MAPE: 0.246676
Imputation RMSE: 31.7366

Running time: 1285 seconds

0.1
15
Total iteration: 56
Tolerance: 9.57584e-05
Imputation MAPE: 0.246281
Imputation RMSE: 32.4707

Running time: 1311 seconds

0.1
20
Total iteration: 57
Tolerance: 8.69084e-05
Imputation MAPE: 0.246779
Imputation RMSE: 32.9351

Running time: 1313 seconds

0.1
25
Total iteration: 57
Tolerance: 9.15024e-05
Imputation MAPE: 0.247411
Imputation RMSE: 33.2977

Running time: 1348 seconds

0.1
30
Total iteration: 57
Tolerance: 9.68404e-05
Imputation MAPE: 0.249756
Imputation RMSE: 33.3353

Running time: 1320 seconds

0.2
5
Total iteration: 56
Tolerance: 8.76805e-05
Imputation MAPE: 0.258506
Imputation RMSE: 35.1232

Running time: 1269 seconds

0.2
10
Total iteration: 56
Tolerance: 9.08054e-05
Imputation MAPE: 0.248095
Imputation RMSE:

In [17]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

# Random Missing (RM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
np.random.seed(1000)
sparse_tensor = mat2ten(dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate), dim, 0)

start = time.time()
time_lags = np.array([1, 2, 3, 4])
alpha = np.ones(3) / 3
rho = 1e-5
lambda0 = 1/5 * rho
theta = 10
epsilon = 1e-4
tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Total iteration: 46
Tolerance: 9.47048e-05
Imputation MAPE: 0.174583
Imputation RMSE: 15.8859

Running time: 892 seconds



In [18]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Block-out Missing (BM)
dense_mat = np.load('../datasets/Portland-data-set/volume.npy')
dim1, dim2 = dense_mat.shape
dim = np.array([dim1, 96, 31])
dense_tensor = mat2ten(dense_mat, dim, 0)
block_window = 4
vec = np.random.rand(int(dim2 / block_window))
temp = np.array([vec] * block_window)
vec = temp.reshape([dim2], order = 'F')
sparse_tensor = mat2ten(dense_mat * np.round(vec + 0.5 - missing_rate)[None, :], dim, 0)

start = time.time()
time_lags = np.array([1, 2, 3, 4])
alpha = np.ones(3) / 3
rho = 1e-5
lambda0 = 1 * rho
theta = 5
epsilon = 1e-4
tensor_hat = latc(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon)
end = time.time()
print('Running time: %d seconds'%(end - start))
print()

Total iteration: 43
Tolerance: 9.35652e-05
Imputation MAPE: 0.240367
Imputation RMSE: 23.5154

Running time: 853 seconds



### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>